In [ ]:
import collections
import heapq
import time
import matplotlib.pyplot as plt

# --- 1. Maze Definition ---
# 0 = Path, 1 = Wall, 2 = Start, 3 = Goal

# Using a 10x10 maze for demonstration
MAZE = [
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    [1, 2, 0, 0, 0, 0, 0, 0, 0, 1],
    [1, 0, 1, 1, 1, 1, 0, 1, 0, 1],
    [1, 0, 0, 0, 0, 1, 0, 1, 0, 1],
    [1, 1, 1, 0, 1, 1, 0, 1, 0, 1],
    [1, 0, 0, 0, 1, 0, 0, 0, 0, 1],
    [1, 0, 1, 1, 1, 0, 1, 1, 0, 1],
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    [1, 0, 1, 1, 1, 1, 1, 1, 3, 1],
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
]

HEIGHT = len(MAZE)
WIDTH = len(MAZE[0])

# Find Start and Goal coordinates
def find_start_goal(maze):
    for r in range(HEIGHT):
        for c in range(WIDTH):
            if maze[r][c] == 2:
                start = (r, c)
            elif maze[r][c] == 3:
                goal = (r, c)
    return start, goal

START, GOAL = find_start_goal(MAZE)

In [2]:
# Function to get neighbors (up, down, left, right)
def get_neighbors(r, c):
    neighbors = []
    # (row, col) offsets for 4-way movement
    for dr, dc in [(0, 1), (0, -1), (1, 0), (-1, 0)]:
        nr, nc = r + dr, c + dc
        if 0 <= nr < HEIGHT and 0 <= nc < WIDTH and MAZE[nr][nc] != 1:
            neighbors.append((nr, nc))
    return neighbors

# Reconstruct the path from the 'parent' map
def reconstruct_path(parent, goal, start):
    path = []
    curr = goal
    while curr != start:
        path.append(curr)
        curr = parent[curr]
    path.append(start)
    return path[::-1] # Reverse the path

# --- Heuristic for A* (Manhattan Distance) ---
def manhattan_distance(p1, p2):
    return abs(p1[0] - p2[0]) + abs(p1[1] - p2[1])

# --- 2a. A* Search Implementation ---
def a_star_search(start, goal):
    # Priority Queue: stores (f_cost, g_cost, (row, col))
    pq = [(0 + manhattan_distance(start, goal), 0, start)] 
    parent = {start: None}
    g_cost = {start: 0}
    nodes_expanded = 0

    while pq:
        nodes_expanded += 1
        f_cost, g, current = heapq.heappop(pq)
        
        if current == goal:
            return reconstruct_path(parent, goal, start), nodes_expanded

        for neighbor in get_neighbors(*current):
            new_g_cost = g + 1 # cost of moving to a neighbor is 1
            
            if neighbor not in g_cost or new_g_cost < g_cost[neighbor]:
                g_cost[neighbor] = new_g_cost
                f = new_g_cost + manhattan_distance(neighbor, goal)
                heapq.heappush(pq, (f, new_g_cost, neighbor))
                parent[neighbor] = current
                
    return None, nodes_expanded # No path found

# --- 2b. BFS Implementation (Queue) ---
def bfs_search(start, goal):
    queue = collections.deque([start])
    parent = {start: None}
    visited = {start}
    nodes_expanded = 0
    
    while queue:
        nodes_expanded += 1
        current = queue.popleft()
        
        if current == goal:
            return reconstruct_path(parent, goal, start), nodes_expanded
        
        for neighbor in get_neighbors(*current):
            if neighbor not in visited:
                visited.add(neighbor)
                parent[neighbor] = current
                queue.append(neighbor)

    return None, nodes_expanded

# --- 2c. DFS Implementation (Stack/Recursion) ---
# Note: Using an explicit stack here for easier performance tracking
def dfs_search(start, goal):
    stack = [start]
    parent = {start: None}
    visited = {start}
    nodes_expanded = 0
    
    while stack:
        nodes_expanded += 1
        current = stack.pop() # LIFO (Stack behavior)
        
        if current == goal:
            return reconstruct_path(parent, goal, start), nodes_expanded
        
        # Explore neighbors in a consistent order (e.g., reverse for visual consistency)
        for neighbor in reversed(get_neighbors(*current)): 
            if neighbor not in visited:
                visited.add(neighbor)
                parent[neighbor] = current
                stack.append(neighbor)

    return None, nodes_expanded


In [ ]:
# --- 3. Execution and Analysis ---

def solve_and_compare(solver_func, name, start, goal):
    start_time = time.time()
    path, nodes = solver_func(start, goal)
    end_time = time.time()
    
    runtime = end_time - start_time
    path_length = len(path) - 1 if path else 0
    
    print(f"\n--- {name} Results ---")
    print(f"Path Found: {'Yes' if path else 'No'}")
    print(f"Path Length (Steps): {path_length}")
    print(f"Nodes Expanded: {nodes}")
    print(f"Execution Time (s): {runtime:.6f}")
    
    return {
        'name': name,
        'path': path,
        'length': path_length,
        'nodes': nodes,
        'time': runtime
    }

def visualize_maze_solution(maze, path, title, nodes_expanded):
    # Create a copy of the maze for visualization
    vis_maze = [row[:] for row in maze]

    # Color mapping for visualization
    # 0=Path (white), 1=Wall (black), 2=Start (green), 3=Goal (red), 4=Solution (blue)
    color_map = {0: 1.0, 1: 0.0, 2: 0.5, 3: 0.7, 4: 0.3} # Using grayscale for simplicity
    
    if path:
        # Mark path in the visualization maze
        for r, c in path:
            if (r, c) != START and (r, c) != GOAL:
                vis_maze[r][c] = 4
    
    # Map numerical maze values to color intensities
    plot_data = [[color_map[cell] for cell in row] for row in vis_maze]

    plt.imshow(plot_data, cmap='viridis') # Using 'viridis' for distinct colors
    plt.title(f'{title}\nPath Length: {len(path)-1 if path else 0}, Nodes Expanded: {nodes_expanded}')
    plt.xticks([]), plt.yticks([])
    plt.colorbar(ticks=[0, 0.3, 0.5, 0.7, 1.0], 
                label='Colors: Black (Wall), Dark Blue (Path), Green (Start), Red (Goal), Light Blue (Solution)')
    plt.show()

if __name__ == "__main__":
    
    solvers = [
        ("A* Search (Manhattan)", a_star_search),
        ("BFS (Shortest Path)", bfs_search),
        ("DFS (Non-Optimal)", dfs_search)
    ]
    
    results = []
    
    # Run and collect results for all algorithms
    for name, func in solvers:
        result = solve_and_compare(func, name, START, GOAL)
        results.append(result)
        
        # Mandatory: Visualization of the solution for each algorithm
        visualize_maze_solution(MAZE, result['path'], name, result['nodes'])
        
    # --- Comparative Study (Results & Analysis) ---
    print("\n--- Comparative Analysis Table ---")
    
    headers = ["Algorithm", "Path Length", "Nodes Expanded", "Time (s)"]
    data = [headers]
    
    for res in results:
        data.append([
            res['name'],
            res['length'],
            res['nodes'],
            f"{res['time']:.6f}"
        ])
        
    # Simple ASCII table display
    col_widths = [max(len(str(item)) for item in col) for col in zip(*data)]
    for row in data:
        print(" | ".join(f"{item:<{width}}" for item, width in zip(row, col_widths)))
    
    # --- Visualization Comparison ---
    # Optional: Plotting nodes expanded
    plt.figure(figsize=(8, 5))
    names = [r['name'] for r in results]
    nodes = [r['nodes'] for r in results]
    plt.bar(names, nodes, color=['skyblue', 'lightcoral', 'lightgreen'])
    plt.title('Comparison of Nodes Expanded')
    plt.ylabel('Nodes Expanded')
    plt.show()

In [ ]:
with open('maze_logic.pkl', 'wb') as f:
    pickle.dump(project_data, f)

print("Pickle file 'maze_logic.pkl' created successfully!")